In [1]:
import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import WikipediaNetwork
from torch_geometric.loader import DataLoader

In [2]:
dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[2277, 2325], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10], adj_t=[2277, 2277, nnz=36101])


In [4]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

10920
7290
4560


# GSAGE

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [6]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [ ]:
idx=[data.train_mask[i][0] for i in range(183)]
train_index = np.where(idx)[0]
print(train_index)

In [7]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 32, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.8193, Train: 48.44%, Valid: 41.56% Test: 42.11%
Run: 01, Epoch: 02, Loss: 1.2171, Train: 54.21%, Valid: 45.82% Test: 47.59%
Run: 01, Epoch: 03, Loss: 1.0920, Train: 55.31%, Valid: 46.50% Test: 47.37%
Run: 01, Epoch: 04, Loss: 1.0285, Train: 57.78%, Valid: 47.60% Test: 48.46%
Run: 01, Epoch: 05, Loss: 0.9675, Train: 64.56%, Valid: 52.67% Test: 51.10%
Run: 01, Epoch: 06, Loss: 0.9117, Train: 70.88%, Valid: 56.10% Test: 54.82%
Run: 01, Epoch: 07, Loss: 0.8672, Train: 74.08%, Valid: 58.30% Test: 57.02%
Run: 01, Epoch: 08, Loss: 0.8248, Train: 76.83%, Valid: 58.44% Test: 57.02%
Run: 01, Epoch: 09, Loss: 0.7855, Train: 78.85%, Valid: 59.53% Test: 57.02%
Run: 01, Epoch: 10, Loss: 0.7368, Train: 81.87%, Valid: 59.81% Test: 56.80%
Run: 01, Epoch: 11, Loss: 0.6989, Train: 84.43%, Valid: 60.08% Test: 56.80%
Run: 01, Epoch: 12, Loss: 0.6507, Train: 85.07%, Valid: 60.91% Test: 58.55%
Run: 01, Epoch: 13, Loss: 0.6257, Train: 85.90%, Valid: 61.04% Test: 57.89%
Run: 01, Epo

Run: 02, Epoch: 07, Loss: 0.8098, Train: 76.92%, Valid: 53.09% Test: 57.89%
Run: 02, Epoch: 08, Loss: 0.7580, Train: 79.03%, Valid: 54.18% Test: 57.89%
Run: 02, Epoch: 09, Loss: 0.7106, Train: 82.51%, Valid: 55.97% Test: 59.87%
Run: 02, Epoch: 10, Loss: 0.6767, Train: 84.43%, Valid: 55.97% Test: 59.43%
Run: 02, Epoch: 11, Loss: 0.6332, Train: 86.08%, Valid: 57.34% Test: 60.75%
Run: 02, Epoch: 12, Loss: 0.5993, Train: 87.64%, Valid: 57.61% Test: 61.84%
Run: 02, Epoch: 13, Loss: 0.5473, Train: 88.28%, Valid: 57.48% Test: 60.09%
Run: 02, Epoch: 14, Loss: 0.5185, Train: 89.56%, Valid: 57.48% Test: 60.96%
Run: 02, Epoch: 15, Loss: 0.5040, Train: 90.75%, Valid: 57.61% Test: 61.62%
Run: 02, Epoch: 16, Loss: 0.4797, Train: 90.66%, Valid: 57.34% Test: 61.40%
Run: 02, Epoch: 17, Loss: 0.4277, Train: 91.76%, Valid: 57.75% Test: 62.50%
Run: 02, Epoch: 18, Loss: 0.4154, Train: 93.59%, Valid: 60.08% Test: 64.25%
Run: 02, Epoch: 19, Loss: 0.4009, Train: 94.87%, Valid: 60.63% Test: 65.13%
Run: 02, Epo

Run: 03, Epoch: 14, Loss: 0.5797, Train: 86.63%, Valid: 58.71% Test: 55.70%
Run: 03, Epoch: 15, Loss: 0.5337, Train: 86.26%, Valid: 58.98% Test: 55.26%
Run: 03, Epoch: 16, Loss: 0.5161, Train: 87.27%, Valid: 59.26% Test: 55.48%
Run: 03, Epoch: 17, Loss: 0.5020, Train: 88.92%, Valid: 59.53% Test: 57.02%
Run: 03, Epoch: 18, Loss: 0.4666, Train: 91.30%, Valid: 60.91% Test: 59.21%
Run: 03, Epoch: 19, Loss: 0.4400, Train: 93.04%, Valid: 61.59% Test: 59.21%
Run: 03, Epoch: 20, Loss: 0.4083, Train: 93.22%, Valid: 61.32% Test: 57.46%
Run: 03, Epoch: 21, Loss: 0.3820, Train: 93.50%, Valid: 61.45% Test: 57.68%
Run: 03, Epoch: 22, Loss: 0.3606, Train: 94.51%, Valid: 62.28% Test: 57.89%
Run: 03, Epoch: 23, Loss: 0.3666, Train: 95.51%, Valid: 62.83% Test: 59.21%
Run: 03, Epoch: 24, Loss: 0.3427, Train: 95.51%, Valid: 62.83% Test: 59.43%
Run: 03, Epoch: 25, Loss: 0.3242, Train: 96.15%, Valid: 62.14% Test: 59.65%
Run: 03, Epoch: 26, Loss: 0.2835, Train: 96.43%, Valid: 62.28% Test: 58.99%
Run: 03, Epo

Run: 04, Epoch: 21, Loss: 0.3786, Train: 93.77%, Valid: 62.55% Test: 64.47%
Run: 04, Epoch: 22, Loss: 0.3552, Train: 94.78%, Valid: 62.83% Test: 64.04%
Run: 04, Epoch: 23, Loss: 0.3495, Train: 95.33%, Valid: 62.41% Test: 64.69%
Run: 04, Epoch: 24, Loss: 0.3254, Train: 95.33%, Valid: 61.87% Test: 65.79%
Run: 04, Epoch: 25, Loss: 0.3157, Train: 96.06%, Valid: 61.32% Test: 65.35%
Run: 04, Epoch: 26, Loss: 0.2867, Train: 95.79%, Valid: 61.73% Test: 65.13%
Run: 04, Epoch: 27, Loss: 0.2749, Train: 96.79%, Valid: 62.28% Test: 65.13%
Run: 04, Epoch: 28, Loss: 0.2839, Train: 97.07%, Valid: 62.00% Test: 64.91%
Run: 04, Epoch: 29, Loss: 0.2456, Train: 97.25%, Valid: 62.14% Test: 64.91%
Run: 04, Epoch: 30, Loss: 0.2413, Train: 97.62%, Valid: 62.83% Test: 65.13%
Run: 04, Epoch: 31, Loss: 0.2357, Train: 97.80%, Valid: 62.55% Test: 65.35%
Run: 04, Epoch: 32, Loss: 0.2257, Train: 97.89%, Valid: 62.00% Test: 65.35%
Run: 04, Epoch: 33, Loss: 0.1900, Train: 97.89%, Valid: 61.45% Test: 64.47%
Run: 04, Epo

Run: 05, Epoch: 28, Loss: 0.2782, Train: 96.89%, Valid: 61.32% Test: 66.01%
Run: 05, Epoch: 29, Loss: 0.2608, Train: 97.16%, Valid: 62.00% Test: 65.57%
Run: 05, Epoch: 30, Loss: 0.2685, Train: 97.34%, Valid: 62.14% Test: 65.79%
Run: 05, Epoch: 31, Loss: 0.2641, Train: 97.89%, Valid: 62.55% Test: 66.45%
Run: 05, Epoch: 32, Loss: 0.2575, Train: 97.99%, Valid: 62.28% Test: 66.67%
Run: 05, Epoch: 33, Loss: 0.2080, Train: 97.80%, Valid: 62.83% Test: 66.89%
Run: 05, Epoch: 34, Loss: 0.2112, Train: 97.80%, Valid: 62.83% Test: 66.67%
Run: 05, Epoch: 35, Loss: 0.1872, Train: 98.08%, Valid: 63.24% Test: 66.67%
Run: 05, Epoch: 36, Loss: 0.1869, Train: 98.17%, Valid: 63.24% Test: 66.45%
Run: 05, Epoch: 37, Loss: 0.1767, Train: 98.26%, Valid: 63.24% Test: 66.45%
Run: 05, Epoch: 38, Loss: 0.1890, Train: 98.17%, Valid: 62.96% Test: 66.45%
Run: 05, Epoch: 39, Loss: 0.1671, Train: 98.35%, Valid: 62.55% Test: 66.23%
Run: 05, Epoch: 40, Loss: 0.1475, Train: 98.72%, Valid: 62.83% Test: 66.01%
Run: 05, Epo

Run: 06, Epoch: 35, Loss: 0.2539, Train: 97.62%, Valid: 63.65% Test: 64.25%
Run: 06, Epoch: 36, Loss: 0.2407, Train: 97.89%, Valid: 63.65% Test: 64.91%
Run: 06, Epoch: 37, Loss: 0.2447, Train: 98.08%, Valid: 63.65% Test: 65.35%
Run: 06, Epoch: 38, Loss: 0.2217, Train: 98.26%, Valid: 63.65% Test: 64.69%
Run: 06, Epoch: 39, Loss: 0.1973, Train: 98.26%, Valid: 63.65% Test: 64.25%
Run: 06, Epoch: 40, Loss: 0.1905, Train: 98.35%, Valid: 63.37% Test: 64.04%
Run: 06, Epoch: 41, Loss: 0.1888, Train: 98.53%, Valid: 63.79% Test: 64.91%
Run: 06, Epoch: 42, Loss: 0.1758, Train: 98.53%, Valid: 63.92% Test: 65.13%
Run: 06, Epoch: 43, Loss: 0.1695, Train: 98.53%, Valid: 63.65% Test: 65.13%
Run: 06, Epoch: 44, Loss: 0.1584, Train: 98.72%, Valid: 63.65% Test: 64.91%
Run: 06, Epoch: 45, Loss: 0.1592, Train: 98.72%, Valid: 63.79% Test: 64.25%
Run: 06, Epoch: 46, Loss: 0.1653, Train: 98.99%, Valid: 64.06% Test: 64.25%
Run: 06, Epoch: 47, Loss: 0.1494, Train: 98.99%, Valid: 64.33% Test: 64.04%
Run: 06, Epo

Run: 07, Epoch: 42, Loss: 0.1515, Train: 98.72%, Valid: 63.79% Test: 60.31%
Run: 07, Epoch: 43, Loss: 0.1623, Train: 98.63%, Valid: 63.51% Test: 60.09%
Run: 07, Epoch: 44, Loss: 0.1648, Train: 98.72%, Valid: 63.37% Test: 61.62%
Run: 07, Epoch: 45, Loss: 0.1294, Train: 99.08%, Valid: 63.92% Test: 61.84%
Run: 07, Epoch: 46, Loss: 0.1275, Train: 99.08%, Valid: 63.92% Test: 61.84%
Run: 07, Epoch: 47, Loss: 0.1484, Train: 99.08%, Valid: 64.20% Test: 62.28%
Run: 07, Epoch: 48, Loss: 0.1236, Train: 99.08%, Valid: 64.20% Test: 62.50%
Run: 07, Epoch: 49, Loss: 0.1292, Train: 99.18%, Valid: 63.92% Test: 62.94%
Run: 07, Epoch: 50, Loss: 0.1040, Train: 99.27%, Valid: 63.79% Test: 62.94%
Run: 07, Epoch: 51, Loss: 0.1149, Train: 99.36%, Valid: 63.79% Test: 62.94%
Run: 07, Epoch: 52, Loss: 0.1225, Train: 99.36%, Valid: 63.37% Test: 63.38%
Run: 07, Epoch: 53, Loss: 0.1065, Train: 99.54%, Valid: 63.37% Test: 62.94%
Run: 07, Epoch: 54, Loss: 0.1025, Train: 99.54%, Valid: 63.79% Test: 62.72%
Run: 07, Epo

Run: 08, Epoch: 49, Loss: 0.1320, Train: 99.18%, Valid: 63.92% Test: 61.40%
Run: 08, Epoch: 50, Loss: 0.1188, Train: 99.36%, Valid: 63.51% Test: 60.96%
Run: 08, Epoch: 51, Loss: 0.1267, Train: 99.36%, Valid: 63.24% Test: 60.53%
Run: 08, Epoch: 52, Loss: 0.1268, Train: 99.45%, Valid: 63.10% Test: 60.96%
Run: 08, Epoch: 53, Loss: 0.1068, Train: 99.45%, Valid: 62.41% Test: 60.96%
Run: 08, Epoch: 54, Loss: 0.1084, Train: 99.63%, Valid: 62.55% Test: 61.40%
Run: 08, Epoch: 55, Loss: 0.1136, Train: 99.63%, Valid: 62.55% Test: 61.18%
Run: 08, Epoch: 56, Loss: 0.0972, Train: 99.45%, Valid: 62.83% Test: 61.84%
Run: 08, Epoch: 57, Loss: 0.1002, Train: 99.54%, Valid: 62.96% Test: 61.40%
Run: 08, Epoch: 58, Loss: 0.1064, Train: 99.54%, Valid: 62.83% Test: 60.96%
Run: 08, Epoch: 59, Loss: 0.0920, Train: 99.54%, Valid: 62.00% Test: 60.31%
Run: 08, Epoch: 60, Loss: 0.0927, Train: 99.54%, Valid: 61.73% Test: 60.53%
Run: 08, Epoch: 61, Loss: 0.0903, Train: 99.63%, Valid: 61.87% Test: 60.31%
Run: 08, Epo

Run: 09, Epoch: 56, Loss: 0.0940, Train: 99.73%, Valid: 62.55% Test: 60.75%
Run: 09, Epoch: 57, Loss: 0.0908, Train: 99.73%, Valid: 62.41% Test: 60.75%
Run: 09, Epoch: 58, Loss: 0.0836, Train: 99.73%, Valid: 62.69% Test: 60.75%
Run: 09, Epoch: 59, Loss: 0.0978, Train: 99.73%, Valid: 63.37% Test: 60.75%
Run: 09, Epoch: 60, Loss: 0.0783, Train: 99.73%, Valid: 62.83% Test: 60.96%
Run: 09, Epoch: 61, Loss: 0.0908, Train: 99.73%, Valid: 63.37% Test: 60.31%
Run: 09, Epoch: 62, Loss: 0.0796, Train: 99.73%, Valid: 63.37% Test: 60.31%
Run: 09, Epoch: 63, Loss: 0.0597, Train: 99.63%, Valid: 63.92% Test: 60.75%
Run: 09, Epoch: 64, Loss: 0.0858, Train: 99.63%, Valid: 64.20% Test: 60.96%
Run: 09, Epoch: 65, Loss: 0.0814, Train: 99.63%, Valid: 64.20% Test: 60.53%
Run: 09, Epoch: 66, Loss: 0.0705, Train: 99.63%, Valid: 64.47% Test: 60.96%
Run: 09, Epoch: 67, Loss: 0.0669, Train: 99.63%, Valid: 63.92% Test: 61.40%
Run: 09, Epoch: 68, Loss: 0.0695, Train: 99.63%, Valid: 64.33% Test: 61.40%
Run: 09, Epo

Run: 10, Epoch: 63, Loss: 0.0523, Train: 99.82%, Valid: 59.53% Test: 60.31%
Run: 10, Epoch: 64, Loss: 0.0546, Train: 99.82%, Valid: 59.81% Test: 60.53%
Run: 10, Epoch: 65, Loss: 0.0525, Train: 99.82%, Valid: 60.22% Test: 60.31%
Run: 10, Epoch: 66, Loss: 0.0549, Train: 99.82%, Valid: 60.63% Test: 59.87%
Run: 10, Epoch: 67, Loss: 0.0513, Train: 99.82%, Valid: 60.49% Test: 60.09%
Run: 10, Epoch: 68, Loss: 0.0565, Train: 99.82%, Valid: 60.36% Test: 60.31%
Run: 10, Epoch: 69, Loss: 0.0550, Train: 99.82%, Valid: 60.49% Test: 60.53%
Run: 10, Epoch: 70, Loss: 0.0647, Train: 99.82%, Valid: 60.22% Test: 60.53%
Run: 10, Epoch: 71, Loss: 0.0433, Train: 99.82%, Valid: 60.22% Test: 60.53%
Run: 10, Epoch: 72, Loss: 0.0536, Train: 99.82%, Valid: 60.22% Test: 60.31%
Run: 10, Epoch: 73, Loss: 0.0639, Train: 99.82%, Valid: 60.22% Test: 60.09%
Run: 10, Epoch: 74, Loss: 0.0519, Train: 99.82%, Valid: 60.49% Test: 59.21%
Run: 10, Epoch: 75, Loss: 0.0524, Train: 99.82%, Valid: 60.49% Test: 59.43%
Run: 10, Epo

# Wise Embedding

In [8]:
dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[2277, 2325], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10], adj_t=[2277, 2277, nnz=36101])


In [9]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,2316,2317,2318,2319,2320,2321,2322,2323,2324,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [11]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.01)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [12]:
def Similarity(array1, array2):
    intersection = np.sum(np.logical_and(array1, array2))
    #union = np.sum(np.logical_or(array1, array2))
    #jaccard_similarity = intersection / union
    
    #return jaccard_similarity
    return intersection

In [ ]:
#It takes long time
Fec=[]
for i in range(23):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=np.zeros(7)
    for j in range(1433):
        for i in range(max(catagories)+1):
            if f[j]==1 and basis[i][j]==1:
                count[i]=count[i]+1;

    for i in range(max(catagories)+1):
        vec.append(count[i])
    f.clear()
    Fec.append(vec)
print(Fec)

In [13]:

Fec=[]
for i in range(Number_nodes):
#for i in range(2):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    vec.append(Similarity(f,basis[0]))
    vec.append(Similarity(f,basis[1]))
    vec.append(Similarity(f,basis[2]))
    vec.append(Similarity(f,basis[3]))
    vec.append(Similarity(f,basis[4]))
    #print(f)
    f.clear()
    Fec.append(vec)
SFec=[]
for i in range(Number_nodes):
#for i in range(2):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    Svec.append(Similarity(f,sel_basis[0]))
    Svec.append(Similarity(f,sel_basis[1]))
    Svec.append(Similarity(f,sel_basis[2]))
    Svec.append(Similarity(f,sel_basis[3]))
    Svec.append(Similarity(f,sel_basis[4]))
    #print(f)
    f.clear()
    SFec.append(Svec)

In [16]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[ 7.,  6.,  6.,  ...,  5.,  5.,  7.],
        [10., 12., 18.,  ...,  8., 16., 15.],
        [19., 24., 39.,  ..., 19., 33., 28.],
        ...,
        [ 3.,  2.,  1.,  ...,  0.,  2.,  2.],
        [16., 20., 32.,  ..., 17., 22., 38.],
        [ 4.,  4.,  3.,  ...,  2.,  1.,  2.]])


'torch.FloatTensor'

# W-GSAGE

In [17]:
data.x=CC_domain
print(data)

Data(x=[2277, 10], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10], adj_t=[2277, 2277, nnz=36101])


In [18]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 32, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.9146, Train: 21.70%, Valid: 23.73% Test: 22.15%
Run: 01, Epoch: 02, Loss: 1.7175, Train: 21.52%, Valid: 20.30% Test: 23.25%
Run: 01, Epoch: 03, Loss: 1.5985, Train: 22.25%, Valid: 19.75% Test: 23.46%
Run: 01, Epoch: 04, Loss: 1.4756, Train: 25.00%, Valid: 22.50% Test: 26.32%
Run: 01, Epoch: 05, Loss: 1.4087, Train: 27.38%, Valid: 26.34% Test: 31.80%
Run: 01, Epoch: 06, Loss: 1.3876, Train: 32.23%, Valid: 30.86% Test: 34.87%
Run: 01, Epoch: 07, Loss: 1.3560, Train: 35.53%, Valid: 34.43% Test: 38.38%
Run: 01, Epoch: 08, Loss: 1.3494, Train: 38.55%, Valid: 36.63% Test: 38.82%
Run: 01, Epoch: 09, Loss: 1.3199, Train: 40.48%, Valid: 37.72% Test: 41.01%
Run: 01, Epoch: 10, Loss: 1.3151, Train: 40.75%, Valid: 37.17% Test: 42.54%
Run: 01, Epoch: 11, Loss: 1.2756, Train: 42.40%, Valid: 37.72% Test: 43.20%
Run: 01, Epoch: 12, Loss: 1.2680, Train: 43.32%, Valid: 38.00% Test: 44.52%
Run: 01, Epoch: 13, Loss: 1.2348, Train: 45.24%, Valid: 41.43% Test: 46.49%
Run: 01, Epo

Run: 02, Epoch: 23, Loss: 1.1189, Train: 55.13%, Valid: 53.50% Test: 55.92%
Run: 02, Epoch: 24, Loss: 1.0965, Train: 56.32%, Valid: 54.73% Test: 58.11%
Run: 02, Epoch: 25, Loss: 1.0642, Train: 57.97%, Valid: 56.10% Test: 60.75%
Run: 02, Epoch: 26, Loss: 1.0769, Train: 57.88%, Valid: 55.42% Test: 59.65%
Run: 02, Epoch: 27, Loss: 1.0390, Train: 58.42%, Valid: 55.01% Test: 59.21%
Run: 02, Epoch: 28, Loss: 1.0468, Train: 57.05%, Valid: 57.75% Test: 59.21%
Run: 02, Epoch: 29, Loss: 1.0255, Train: 58.15%, Valid: 58.30% Test: 59.43%
Run: 02, Epoch: 30, Loss: 1.0240, Train: 60.16%, Valid: 58.30% Test: 61.18%
Run: 02, Epoch: 31, Loss: 1.0120, Train: 61.54%, Valid: 59.12% Test: 62.06%
Run: 02, Epoch: 32, Loss: 0.9934, Train: 62.36%, Valid: 61.59% Test: 62.72%
Run: 02, Epoch: 33, Loss: 0.9768, Train: 61.17%, Valid: 62.00% Test: 63.38%
Run: 02, Epoch: 34, Loss: 0.9624, Train: 60.71%, Valid: 62.00% Test: 62.72%
Run: 02, Epoch: 35, Loss: 0.9813, Train: 63.10%, Valid: 62.14% Test: 63.60%
Run: 02, Epo

Run: 03, Epoch: 48, Loss: 0.8945, Train: 69.41%, Valid: 61.45% Test: 63.16%
Run: 03, Epoch: 49, Loss: 0.9205, Train: 71.89%, Valid: 63.92% Test: 64.91%
Run: 03, Epoch: 50, Loss: 0.8772, Train: 72.71%, Valid: 64.88% Test: 64.69%
Run: 03, Epoch: 51, Loss: 0.8871, Train: 71.79%, Valid: 66.39% Test: 65.79%
Run: 03, Epoch: 52, Loss: 0.8674, Train: 71.52%, Valid: 66.53% Test: 66.23%
Run: 03, Epoch: 53, Loss: 0.8825, Train: 72.99%, Valid: 65.57% Test: 66.89%
Run: 03, Epoch: 54, Loss: 0.8628, Train: 71.61%, Valid: 65.43% Test: 66.89%
Run: 03, Epoch: 55, Loss: 0.8530, Train: 73.90%, Valid: 66.53% Test: 66.67%
Run: 03, Epoch: 56, Loss: 0.8419, Train: 73.08%, Valid: 67.76% Test: 66.45%
Run: 03, Epoch: 57, Loss: 0.8363, Train: 73.44%, Valid: 65.98% Test: 64.91%
Run: 03, Epoch: 58, Loss: 0.8626, Train: 74.82%, Valid: 68.18% Test: 65.79%
Run: 03, Epoch: 59, Loss: 0.8127, Train: 77.29%, Valid: 69.27% Test: 69.08%
Run: 03, Epoch: 60, Loss: 0.7900, Train: 78.21%, Valid: 70.23% Test: 70.18%
Run: 03, Epo

Run: 04, Epoch: 75, Loss: 0.7546, Train: 75.64%, Valid: 73.11% Test: 75.44%
Run: 04, Epoch: 76, Loss: 0.7684, Train: 75.92%, Valid: 73.66% Test: 75.66%
Run: 04, Epoch: 77, Loss: 0.7957, Train: 76.28%, Valid: 72.70% Test: 76.32%
Run: 04, Epoch: 78, Loss: 0.7673, Train: 76.10%, Valid: 70.92% Test: 75.44%
Run: 04, Epoch: 79, Loss: 0.7739, Train: 75.64%, Valid: 71.74% Test: 73.90%
Run: 04, Epoch: 80, Loss: 0.7790, Train: 76.83%, Valid: 71.60% Test: 73.25%
Run: 04, Epoch: 81, Loss: 0.7418, Train: 76.37%, Valid: 71.06% Test: 73.46%
Run: 04, Epoch: 82, Loss: 0.7541, Train: 76.47%, Valid: 74.21% Test: 75.22%
Run: 04, Epoch: 83, Loss: 0.7639, Train: 76.56%, Valid: 73.39% Test: 76.75%
Run: 04, Epoch: 84, Loss: 0.7512, Train: 77.56%, Valid: 75.03% Test: 77.19%
Run: 04, Epoch: 85, Loss: 0.7633, Train: 78.11%, Valid: 75.58% Test: 78.95%
Run: 04, Epoch: 86, Loss: 0.7437, Train: 78.11%, Valid: 74.35% Test: 78.51%
Run: 04, Epoch: 87, Loss: 0.7190, Train: 77.20%, Valid: 72.15% Test: 77.41%
Run: 04, Epo

Run: 06, Epoch: 01, Loss: 1.7793, Train: 19.05%, Valid: 19.07% Test: 20.39%
Run: 06, Epoch: 02, Loss: 1.6061, Train: 24.18%, Valid: 24.28% Test: 25.00%
Run: 06, Epoch: 03, Loss: 1.4736, Train: 26.56%, Valid: 26.06% Test: 29.39%
Run: 06, Epoch: 04, Loss: 1.4375, Train: 26.19%, Valid: 27.71% Test: 30.04%
Run: 06, Epoch: 05, Loss: 1.3963, Train: 30.04%, Valid: 31.96% Test: 35.53%
Run: 06, Epoch: 06, Loss: 1.3974, Train: 36.08%, Valid: 36.63% Test: 39.91%
Run: 06, Epoch: 07, Loss: 1.3449, Train: 43.04%, Valid: 42.94% Test: 44.74%
Run: 06, Epoch: 08, Loss: 1.3514, Train: 48.90%, Valid: 46.91% Test: 46.05%
Run: 06, Epoch: 09, Loss: 1.3116, Train: 51.37%, Valid: 47.87% Test: 46.93%
Run: 06, Epoch: 10, Loss: 1.2945, Train: 50.37%, Valid: 48.56% Test: 48.68%
Run: 06, Epoch: 11, Loss: 1.2816, Train: 49.27%, Valid: 49.11% Test: 50.44%
Run: 06, Epoch: 12, Loss: 1.2752, Train: 51.37%, Valid: 49.52% Test: 53.07%
Run: 06, Epoch: 13, Loss: 1.2349, Train: 51.83%, Valid: 50.75% Test: 53.29%
Run: 06, Epo

Run: 07, Epoch: 28, Loss: 1.0075, Train: 64.47%, Valid: 59.95% Test: 61.40%
Run: 07, Epoch: 29, Loss: 1.0264, Train: 64.56%, Valid: 59.81% Test: 60.75%
Run: 07, Epoch: 30, Loss: 1.0109, Train: 64.01%, Valid: 59.95% Test: 61.62%
Run: 07, Epoch: 31, Loss: 0.9790, Train: 63.00%, Valid: 59.67% Test: 61.18%
Run: 07, Epoch: 32, Loss: 0.9735, Train: 62.82%, Valid: 59.26% Test: 59.87%
Run: 07, Epoch: 33, Loss: 0.9607, Train: 61.08%, Valid: 59.53% Test: 59.43%
Run: 07, Epoch: 34, Loss: 0.9453, Train: 61.45%, Valid: 59.53% Test: 60.09%
Run: 07, Epoch: 35, Loss: 0.9330, Train: 62.64%, Valid: 59.53% Test: 61.18%
Run: 07, Epoch: 36, Loss: 0.9147, Train: 62.73%, Valid: 59.53% Test: 61.84%
Run: 07, Epoch: 37, Loss: 0.9265, Train: 62.91%, Valid: 60.22% Test: 60.96%
Run: 07, Epoch: 38, Loss: 0.9327, Train: 63.46%, Valid: 61.45% Test: 63.60%
Run: 07, Epoch: 39, Loss: 0.8972, Train: 65.20%, Valid: 62.96% Test: 66.01%
Run: 07, Epoch: 40, Loss: 0.8738, Train: 67.95%, Valid: 64.61% Test: 66.89%
Run: 07, Epo

Run: 08, Epoch: 55, Loss: 0.8686, Train: 73.81%, Valid: 71.06% Test: 66.23%
Run: 08, Epoch: 56, Loss: 0.8983, Train: 73.72%, Valid: 70.51% Test: 66.45%
Run: 08, Epoch: 57, Loss: 0.8425, Train: 71.43%, Valid: 68.86% Test: 67.54%
Run: 08, Epoch: 58, Loss: 0.8632, Train: 68.22%, Valid: 65.98% Test: 66.45%
Run: 08, Epoch: 59, Loss: 0.8369, Train: 70.97%, Valid: 67.90% Test: 66.67%
Run: 08, Epoch: 60, Loss: 0.8570, Train: 72.25%, Valid: 69.00% Test: 64.69%
Run: 08, Epoch: 61, Loss: 0.8411, Train: 72.07%, Valid: 69.82% Test: 65.35%
Run: 08, Epoch: 62, Loss: 0.8466, Train: 71.79%, Valid: 69.27% Test: 64.91%
Run: 08, Epoch: 63, Loss: 0.8412, Train: 71.89%, Valid: 68.18% Test: 66.23%
Run: 08, Epoch: 64, Loss: 0.8082, Train: 73.35%, Valid: 69.27% Test: 68.64%
Run: 08, Epoch: 65, Loss: 0.8206, Train: 73.35%, Valid: 69.82% Test: 68.86%
Run: 08, Epoch: 66, Loss: 0.8555, Train: 73.53%, Valid: 70.10% Test: 66.45%
Run: 08, Epoch: 67, Loss: 0.7956, Train: 75.92%, Valid: 72.29% Test: 66.01%
Run: 08, Epo

Run: 09, Epoch: 62, Loss: 0.8287, Train: 74.36%, Valid: 71.60% Test: 73.46%
Run: 09, Epoch: 63, Loss: 0.7769, Train: 74.54%, Valid: 71.74% Test: 73.03%
Run: 09, Epoch: 64, Loss: 0.8363, Train: 72.25%, Valid: 70.51% Test: 70.18%
Run: 09, Epoch: 65, Loss: 0.8044, Train: 71.79%, Valid: 70.92% Test: 68.20%
Run: 09, Epoch: 66, Loss: 0.7831, Train: 74.73%, Valid: 73.80% Test: 71.93%
Run: 09, Epoch: 67, Loss: 0.8016, Train: 77.29%, Valid: 76.13% Test: 73.90%
Run: 09, Epoch: 68, Loss: 0.7961, Train: 76.83%, Valid: 75.72% Test: 74.34%
Run: 09, Epoch: 69, Loss: 0.7755, Train: 77.47%, Valid: 75.31% Test: 74.34%
Run: 09, Epoch: 70, Loss: 0.7983, Train: 74.27%, Valid: 72.29% Test: 73.25%
Run: 09, Epoch: 71, Loss: 0.8024, Train: 72.71%, Valid: 71.06% Test: 70.18%
Run: 09, Epoch: 72, Loss: 0.8099, Train: 77.29%, Valid: 73.94% Test: 74.12%
Run: 09, Epoch: 73, Loss: 0.7783, Train: 77.20%, Valid: 74.49% Test: 73.90%
Run: 09, Epoch: 74, Loss: 0.7924, Train: 74.45%, Valid: 73.11% Test: 73.68%
Run: 09, Epo

Run: 10, Epoch: 89, Loss: 0.7117, Train: 78.75%, Valid: 76.82% Test: 75.66%
Run: 10, Epoch: 90, Loss: 0.7388, Train: 78.66%, Valid: 76.13% Test: 75.00%
Run: 10, Epoch: 91, Loss: 0.7111, Train: 78.85%, Valid: 77.23% Test: 75.00%
Run: 10, Epoch: 92, Loss: 0.6955, Train: 79.30%, Valid: 77.50% Test: 75.44%
Run: 10, Epoch: 93, Loss: 0.6925, Train: 79.58%, Valid: 77.91% Test: 75.88%
Run: 10, Epoch: 94, Loss: 0.7191, Train: 79.40%, Valid: 78.19% Test: 76.54%
Run: 10, Epoch: 95, Loss: 0.6913, Train: 78.48%, Valid: 77.50% Test: 76.10%
Run: 10, Epoch: 96, Loss: 0.7213, Train: 78.30%, Valid: 76.68% Test: 76.32%
Run: 10, Epoch: 97, Loss: 0.6968, Train: 77.84%, Valid: 76.54% Test: 76.32%
Run: 10, Epoch: 98, Loss: 0.6764, Train: 78.48%, Valid: 78.19% Test: 75.66%
Run: 10, Epoch: 99, Loss: 0.6610, Train: 78.11%, Valid: 78.46% Test: 76.10%
Run: 10, Epoch: 100, Loss: 0.6785, Train: 78.30%, Valid: 78.74% Test: 76.10%
Run 10:
Highest Train: 79.58
Highest Valid: 78.74
  Final Train: 78.30
   Final Test: 7

# Topological Encodding 

In [4]:
dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon')
data = dataset[0]
print(data)

Data(x=[2277, 2325], edge_index=[2, 36101], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10])


In [22]:
print(data.edge_index.numpy())

[[   0    0    0 ... 2276 2276 2276]
 [1161 1667 1991 ... 2212 2246 2276]]


In [23]:
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [24]:
# a "plain" graph is undirected
#G = nx.DiGraph()
G = nx.Graph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [25]:
print(G.number_of_edges())

31421


In [26]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [27]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [28]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

1  |  109 

2  |  89 

3  |  94 

4  |  167 

5  |  100 

6  |  96 

7  |  121 

8  |  81 

9  |  66 

10  |  99 

11  |  69 

12  |  54 

13  |  62 

14  |  38 

15  |  19 

16  |  39 

17  |  25 

18  |  131 

19  |  17 

20  |  34 

21  |  5 

22  |  8 

23  |  33 

24  |  28 

25  |  54 

26  |  3 

27  |  6 

28  |  28 

29  |  36 

30  |  23 

31  |  30 

32  |  9 

33  |  60 

34  |  3 

35  |  6 

37  |  6 

38  |  12 

39  |  4 

40  |  9 

41  |  20 

42  |  2 

43  |  4 

44  |  6 

45  |  1 

46  |  6 

47  |  2 

48  |  8 

49  |  3 

50  |  4 

51  |  19 

52  |  17 

53  |  6 

54  |  3 

55  |  2 

56  |  9 

57  |  14 

59  |  2 

60  |  6 

61  |  7 

62  |  6 

63  |  6 

64  |  2 

65  |  5 

66  |  4 

67  |  4 

68  |  5 

69  |  2 

70  |  2 

71  |  12 

72  |  10 

73  |  3 

74  |  6 

75  |  2 

76  |  5 

77  |  2 

78  |  8 

79  |  3 

80  |  5 

81  |  2 

82  |  1 

83  |  1 

85  |  1 

86  |  1 

87  |  2 

89  |  1 

91  |  1 

92  |  2 

93  |  1 

9

In [29]:
import pyflagser
Node_fil=[1,2,3,4,5,6,7,8,9,10,12,15,20,25,30,50,100,200,400]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=True, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 45 (1%)

KeyboardInterrupt: 

In [6]:
import pandas as pd
data = pd.read_csv('Feature_Cham.csv')
data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,9.1,10.1,11.1,12.1,13.1,14.1,15.1,16.1,17.1,18.1
0,0,0,5,5,13,14,14,14,8,8,...,3,5,13,13,10,10,10,10,10,10
1,1,4,13,20,25,27,24,25,27,26,...,0,0,3,13,40,56,25,3,3,3
2,2,3,4,11,12,17,18,26,26,30,...,0,4,6,13,23,38,170,308,44,13
3,3,7,7,7,7,7,7,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,4,33,89,109,134,201,266,309,324,328,...,14,15,27,53,105,185,1191,1169,617,617


In [7]:
Data1=data.drop(['Unnamed: 0'], axis=1)
Data1.head()

,0,1,2,3,4,5,6,7,8,9,...,9.1,10.1,11.1,12.1,13.1,14.1,15.1,16.1,17.1,18.1
0,0,5,5,13,14,14,14,8,8,8,...,3,5,13,13,10,10,10,10,10,10
1,4,13,20,25,27,24,25,27,26,27,...,0,0,3,13,40,56,25,3,3,3
2,3,4,11,12,17,18,26,26,30,31,...,0,4,6,13,23,38,170,308,44,13
3,7,7,7,7,7,7,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,33,89,109,134,201,266,309,324,328,321,...,14,15,27,53,105,185,1191,1169,617,617


In [8]:
Topo_fe=torch.tensor(Data1.values).float()

In [36]:
dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[2277, 2325], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10], adj_t=[2277, 2277, nnz=36101])


In [37]:
data.x=CC_domain
data.topo=Topo_fe
print(data)

Data(x=[2277, 10], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10], adj_t=[2277, 2277, nnz=36101], topo=[2277, 38])


# TOPO-W-GSAGE

In [9]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x
        #return x.log_softmax(dim=-1)

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [41]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 32, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.01)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.6141, Train: 20.05%, Valid: 20.30% Test: 19.08%
Run: 01, Epoch: 02, Loss: 1.5395, Train: 22.07%, Valid: 24.42% Test: 22.37%
Run: 01, Epoch: 03, Loss: 1.3991, Train: 22.07%, Valid: 24.42% Test: 22.37%
Run: 01, Epoch: 04, Loss: 1.3348, Train: 22.07%, Valid: 24.42% Test: 22.37%
Run: 01, Epoch: 05, Loss: 1.3077, Train: 22.07%, Valid: 24.42% Test: 22.37%
Run: 01, Epoch: 06, Loss: 1.2708, Train: 25.27%, Valid: 26.89% Test: 25.66%
Run: 01, Epoch: 07, Loss: 1.2461, Train: 29.49%, Valid: 30.32% Test: 29.61%
Run: 01, Epoch: 08, Loss: 1.2377, Train: 31.23%, Valid: 32.10% Test: 32.24%
Run: 01, Epoch: 09, Loss: 1.1869, Train: 30.13%, Valid: 31.82% Test: 30.92%
Run: 01, Epoch: 10, Loss: 1.1607, Train: 33.42%, Valid: 35.53% Test: 34.65%
Run: 01, Epoch: 11, Loss: 1.1692, Train: 35.16%, Valid: 36.90% Test: 38.16%
Run: 01, Epoch: 12, Loss: 1.1222, Train: 39.19%, Valid: 41.70% Test: 39.69%
Run: 01, Epoch: 13, Loss: 1.1180, Train: 42.77%, Valid: 44.99% Test: 41.89%
Run: 01, Epo

Run: 02, Epoch: 11, Loss: 1.1470, Train: 46.79%, Valid: 45.54% Test: 47.37%
Run: 02, Epoch: 12, Loss: 1.1332, Train: 45.79%, Valid: 46.78% Test: 45.61%
Run: 02, Epoch: 13, Loss: 1.1032, Train: 44.87%, Valid: 44.86% Test: 41.89%
Run: 02, Epoch: 14, Loss: 1.0867, Train: 37.55%, Valid: 38.27% Test: 36.40%
Run: 02, Epoch: 15, Loss: 1.0839, Train: 49.36%, Valid: 47.33% Test: 46.49%
Run: 02, Epoch: 16, Loss: 1.0772, Train: 55.77%, Valid: 53.36% Test: 54.39%
Run: 02, Epoch: 17, Loss: 1.0348, Train: 57.88%, Valid: 53.64% Test: 54.61%
Run: 02, Epoch: 18, Loss: 1.0184, Train: 58.24%, Valid: 54.46% Test: 53.29%
Run: 02, Epoch: 19, Loss: 1.0107, Train: 59.71%, Valid: 58.02% Test: 57.89%
Run: 02, Epoch: 20, Loss: 0.9894, Train: 57.14%, Valid: 57.48% Test: 55.26%
Run: 02, Epoch: 21, Loss: 0.9619, Train: 58.70%, Valid: 57.06% Test: 56.36%
Run: 02, Epoch: 22, Loss: 0.9687, Train: 60.99%, Valid: 60.49% Test: 59.65%
Run: 02, Epoch: 23, Loss: 0.9608, Train: 59.52%, Valid: 58.44% Test: 59.43%
Run: 02, Epo

Run: 03, Epoch: 21, Loss: 0.9451, Train: 62.27%, Valid: 56.93% Test: 54.61%
Run: 03, Epoch: 22, Loss: 0.9195, Train: 63.00%, Valid: 57.06% Test: 57.24%
Run: 03, Epoch: 23, Loss: 0.9256, Train: 63.92%, Valid: 57.61% Test: 58.33%
Run: 03, Epoch: 24, Loss: 0.8980, Train: 65.75%, Valid: 58.02% Test: 60.31%
Run: 03, Epoch: 25, Loss: 0.8384, Train: 65.75%, Valid: 58.98% Test: 59.65%
Run: 03, Epoch: 26, Loss: 0.8840, Train: 66.94%, Valid: 60.91% Test: 59.43%
Run: 03, Epoch: 27, Loss: 0.8641, Train: 70.42%, Valid: 62.69% Test: 62.94%
Run: 03, Epoch: 28, Loss: 0.8630, Train: 69.51%, Valid: 63.65% Test: 62.50%
Run: 03, Epoch: 29, Loss: 0.8275, Train: 71.34%, Valid: 66.67% Test: 64.69%
Run: 03, Epoch: 30, Loss: 0.8470, Train: 72.89%, Valid: 67.90% Test: 66.23%
Run: 03, Epoch: 31, Loss: 0.8218, Train: 73.81%, Valid: 67.90% Test: 65.57%
Run: 03, Epoch: 32, Loss: 0.8210, Train: 72.80%, Valid: 66.12% Test: 64.25%
Run: 03, Epoch: 33, Loss: 0.7990, Train: 70.97%, Valid: 64.33% Test: 64.47%
Run: 03, Epo

Run: 04, Epoch: 31, Loss: 0.8913, Train: 67.95%, Valid: 64.88% Test: 69.96%
Run: 04, Epoch: 32, Loss: 0.9166, Train: 71.34%, Valid: 66.94% Test: 74.78%
Run: 04, Epoch: 33, Loss: 0.8760, Train: 71.98%, Valid: 65.71% Test: 72.81%
Run: 04, Epoch: 34, Loss: 0.8636, Train: 68.41%, Valid: 63.51% Test: 73.25%
Run: 04, Epoch: 35, Loss: 0.8649, Train: 66.94%, Valid: 63.65% Test: 70.83%
Run: 04, Epoch: 36, Loss: 0.8933, Train: 68.32%, Valid: 65.98% Test: 70.83%
Run: 04, Epoch: 37, Loss: 0.8543, Train: 69.87%, Valid: 67.35% Test: 71.27%
Run: 04, Epoch: 38, Loss: 0.8331, Train: 72.89%, Valid: 68.86% Test: 77.63%
Run: 04, Epoch: 39, Loss: 0.8170, Train: 73.99%, Valid: 69.41% Test: 75.22%
Run: 04, Epoch: 40, Loss: 0.8266, Train: 71.52%, Valid: 66.12% Test: 71.71%
Run: 04, Epoch: 41, Loss: 0.8096, Train: 71.25%, Valid: 63.65% Test: 70.39%
Run: 04, Epoch: 42, Loss: 0.7938, Train: 72.89%, Valid: 66.53% Test: 71.49%
Run: 04, Epoch: 43, Loss: 0.7934, Train: 72.62%, Valid: 65.84% Test: 72.59%
Run: 04, Epo

Run: 05, Epoch: 40, Loss: 0.8222, Train: 74.08%, Valid: 69.96% Test: 71.27%
Run: 05, Epoch: 41, Loss: 0.8917, Train: 74.08%, Valid: 68.59% Test: 71.49%
Run: 05, Epoch: 42, Loss: 0.8006, Train: 73.63%, Valid: 69.55% Test: 68.42%
Run: 05, Epoch: 43, Loss: 0.7890, Train: 74.91%, Valid: 71.19% Test: 69.74%
Run: 05, Epoch: 44, Loss: 0.7946, Train: 73.26%, Valid: 69.82% Test: 68.42%
Run: 05, Epoch: 45, Loss: 0.7999, Train: 69.87%, Valid: 65.71% Test: 67.11%
Run: 05, Epoch: 46, Loss: 0.8035, Train: 72.53%, Valid: 69.00% Test: 70.18%
Run: 05, Epoch: 47, Loss: 0.7561, Train: 73.81%, Valid: 71.47% Test: 73.03%
Run: 05, Epoch: 48, Loss: 0.8143, Train: 73.72%, Valid: 72.98% Test: 71.93%
Run: 05, Epoch: 49, Loss: 0.8117, Train: 74.18%, Valid: 73.53% Test: 73.68%
Run: 05, Epoch: 50, Loss: 0.7761, Train: 74.73%, Valid: 72.57% Test: 72.37%
Run: 05, Epoch: 51, Loss: 0.8130, Train: 72.53%, Valid: 68.18% Test: 68.64%
Run: 05, Epoch: 52, Loss: 0.7647, Train: 70.70%, Valid: 68.18% Test: 67.11%
Run: 05, Epo

Run: 06, Epoch: 53, Loss: 0.7508, Train: 79.30%, Valid: 73.80% Test: 75.22%
Run: 06, Epoch: 54, Loss: 0.6968, Train: 79.30%, Valid: 73.94% Test: 76.97%
Run: 06, Epoch: 55, Loss: 0.6947, Train: 77.20%, Valid: 73.39% Test: 76.97%
Run: 06, Epoch: 56, Loss: 0.7344, Train: 76.83%, Valid: 73.11% Test: 76.54%
Run: 06, Epoch: 57, Loss: 0.6946, Train: 79.21%, Valid: 74.21% Test: 76.75%
Run: 06, Epoch: 58, Loss: 0.7161, Train: 80.68%, Valid: 74.90% Test: 74.78%
Run: 06, Epoch: 59, Loss: 0.6933, Train: 80.77%, Valid: 73.11% Test: 76.10%
Run: 06, Epoch: 60, Loss: 0.6789, Train: 80.77%, Valid: 72.84% Test: 75.66%
Run: 06, Epoch: 61, Loss: 0.7112, Train: 81.87%, Valid: 74.21% Test: 76.97%
Run: 06, Epoch: 62, Loss: 0.6625, Train: 81.78%, Valid: 74.90% Test: 77.85%
Run: 06, Epoch: 63, Loss: 0.6906, Train: 79.95%, Valid: 73.39% Test: 78.73%
Run: 06, Epoch: 64, Loss: 0.6908, Train: 80.40%, Valid: 73.53% Test: 78.07%
Run: 06, Epoch: 65, Loss: 0.6860, Train: 81.32%, Valid: 74.07% Test: 77.85%
Run: 06, Epo

Run: 07, Epoch: 68, Loss: 0.6313, Train: 80.59%, Valid: 73.80% Test: 77.41%
Run: 07, Epoch: 69, Loss: 0.6456, Train: 80.86%, Valid: 73.11% Test: 77.41%
Run: 07, Epoch: 70, Loss: 0.6613, Train: 80.68%, Valid: 74.76% Test: 76.54%
Run: 07, Epoch: 71, Loss: 0.6504, Train: 83.15%, Valid: 78.74% Test: 80.92%
Run: 07, Epoch: 72, Loss: 0.6315, Train: 79.03%, Valid: 76.68% Test: 79.61%
Run: 07, Epoch: 73, Loss: 0.6568, Train: 78.66%, Valid: 75.72% Test: 78.95%
Run: 07, Epoch: 74, Loss: 0.6655, Train: 83.79%, Valid: 78.46% Test: 81.14%
Run: 07, Epoch: 75, Loss: 0.6463, Train: 82.05%, Valid: 75.72% Test: 80.48%
Run: 07, Epoch: 76, Loss: 0.6379, Train: 80.40%, Valid: 74.90% Test: 77.85%
Run: 07, Epoch: 77, Loss: 0.6155, Train: 80.04%, Valid: 73.39% Test: 77.63%
Run: 07, Epoch: 78, Loss: 0.6278, Train: 81.23%, Valid: 74.62% Test: 78.51%
Run: 07, Epoch: 79, Loss: 0.6668, Train: 81.68%, Valid: 75.86% Test: 78.73%
Run: 07, Epoch: 80, Loss: 0.6033, Train: 82.05%, Valid: 75.99% Test: 78.07%
Run: 07, Epo

Run: 08, Epoch: 77, Loss: 0.6574, Train: 83.24%, Valid: 76.82% Test: 75.22%
Run: 08, Epoch: 78, Loss: 0.6381, Train: 82.78%, Valid: 76.54% Test: 74.12%
Run: 08, Epoch: 79, Loss: 0.6307, Train: 82.33%, Valid: 75.17% Test: 74.34%
Run: 08, Epoch: 80, Loss: 0.6345, Train: 82.51%, Valid: 74.21% Test: 76.10%
Run: 08, Epoch: 81, Loss: 0.6098, Train: 80.95%, Valid: 74.07% Test: 76.54%
Run: 08, Epoch: 82, Loss: 0.6589, Train: 81.41%, Valid: 74.62% Test: 76.75%
Run: 08, Epoch: 83, Loss: 0.6440, Train: 82.97%, Valid: 76.13% Test: 76.32%
Run: 08, Epoch: 84, Loss: 0.6390, Train: 83.24%, Valid: 77.50% Test: 75.44%
Run: 08, Epoch: 85, Loss: 0.6267, Train: 82.05%, Valid: 75.03% Test: 75.66%
Run: 08, Epoch: 86, Loss: 0.6325, Train: 82.42%, Valid: 77.09% Test: 75.22%
Run: 08, Epoch: 87, Loss: 0.6145, Train: 83.42%, Valid: 78.33% Test: 76.97%
Run: 08, Epoch: 88, Loss: 0.6152, Train: 84.34%, Valid: 76.54% Test: 74.34%
Run: 08, Epoch: 89, Loss: 0.6009, Train: 83.06%, Valid: 75.31% Test: 74.56%
Run: 08, Epo

Run: 09, Epoch: 84, Loss: 0.6561, Train: 82.14%, Valid: 78.33% Test: 74.78%
Run: 09, Epoch: 85, Loss: 0.6233, Train: 80.77%, Valid: 76.95% Test: 76.32%
Run: 09, Epoch: 86, Loss: 0.6494, Train: 76.56%, Valid: 70.92% Test: 73.03%
Run: 09, Epoch: 87, Loss: 0.6184, Train: 74.08%, Valid: 69.68% Test: 72.37%
Run: 09, Epoch: 88, Loss: 0.6872, Train: 77.47%, Valid: 72.43% Test: 72.59%
Run: 09, Epoch: 89, Loss: 0.6196, Train: 80.40%, Valid: 76.41% Test: 75.22%
Run: 09, Epoch: 90, Loss: 0.6189, Train: 80.86%, Valid: 78.19% Test: 77.63%
Run: 09, Epoch: 91, Loss: 0.6720, Train: 81.87%, Valid: 78.74% Test: 76.32%
Run: 09, Epoch: 92, Loss: 0.6746, Train: 80.86%, Valid: 76.95% Test: 74.78%
Run: 09, Epoch: 93, Loss: 0.6575, Train: 80.77%, Valid: 77.23% Test: 73.90%
Run: 09, Epoch: 94, Loss: 0.6311, Train: 81.87%, Valid: 77.78% Test: 76.75%
Run: 09, Epoch: 95, Loss: 0.6354, Train: 82.88%, Valid: 78.88% Test: 76.75%
Run: 09, Epoch: 96, Loss: 0.5926, Train: 81.87%, Valid: 77.50% Test: 75.88%
Run: 09, Epo

Run: 10, Epoch: 91, Loss: 0.6705, Train: 82.42%, Valid: 78.60% Test: 76.75%
Run: 10, Epoch: 92, Loss: 0.6677, Train: 82.78%, Valid: 77.78% Test: 78.73%
Run: 10, Epoch: 93, Loss: 0.6958, Train: 82.97%, Valid: 78.46% Test: 79.39%
Run: 10, Epoch: 94, Loss: 0.6247, Train: 82.69%, Valid: 79.29% Test: 78.07%
Run: 10, Epoch: 95, Loss: 0.6501, Train: 81.32%, Valid: 79.56% Test: 78.29%
Run: 10, Epoch: 96, Loss: 0.6595, Train: 81.87%, Valid: 77.37% Test: 76.10%
Run: 10, Epoch: 97, Loss: 0.6349, Train: 81.96%, Valid: 78.46% Test: 79.39%
Run: 10, Epoch: 98, Loss: 0.6504, Train: 81.68%, Valid: 79.97% Test: 80.04%
Run: 10, Epoch: 99, Loss: 0.6137, Train: 82.88%, Valid: 80.25% Test: 79.82%
Run: 10, Epoch: 100, Loss: 0.6099, Train: 81.87%, Valid: 79.70% Test: 78.29%
Run 10:
Highest Train: 82.97
Highest Valid: 80.25
  Final Train: 82.88
   Final Test: 79.82
All runs:
Highest Train: 83.60 ± 1.17
Highest Valid: 78.88 ± 1.04
  Final Train: 82.93 ± 1.04
   Final Test: 78.14 ± 2.18


# TOPO-GSAGE

In [10]:
dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon',transform=T.ToSparseTensor())
data = dataset[0]
data.topo=Topo_fe
print(data)

Data(x=[2277, 2325], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10], adj_t=[2277, 2277, nnz=36101], topo=[2277, 38])


In [12]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 32, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.01)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                #print(f'Run: {run + 1:02d}, 'f'Epoch: {epoch:02d}, ' f'Loss: {loss:.4f}, 'f'Train: {100 * train_acc:.2f}%, '
                #      f'Valid: {100 * valid_acc:.2f}% '
                #      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run 01:
Highest Train: 100.00
Highest Valid: 68.31
  Final Train: 99.18
   Final Test: 65.13
Run 02:
Highest Train: 100.00
Highest Valid: 62.69
  Final Train: 99.82
   Final Test: 68.20
Run 03:
Highest Train: 100.00
Highest Valid: 68.45
  Final Train: 99.82
   Final Test: 63.60
Run 04:
Highest Train: 99.91
Highest Valid: 65.84
  Final Train: 99.82
   Final Test: 66.45
Run 05:
Highest Train: 99.91
Highest Valid: 66.80
  Final Train: 99.91
   Final Test: 65.57
Run 06:
Highest Train: 100.00
Highest Valid: 66.80
  Final Train: 100.00
   Final Test: 66.67
Run 07:
Highest Train: 100.00
Highest Valid: 65.98
  Final Train: 97.89
   Final Test: 64.04
Run 08:
Highest Train: 100.00
Highest Valid: 65.84
  Final Train: 98.90
   Final Test: 65.13
Run 09:
Highest Train: 99.91
Highest Valid: 67.08
  Final Train: 99.91
   Final Test: 64.25
Run 10:
Highest Train: 99.82
Highest Valid: 65.02
  Final Train: 99.63
   Final Test: 66.01
All runs:
Highest Train: 99.95 ± 0.06
Highest Valid: 66.28 ± 1.67
  Final